In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import os
from tqdm import tqdm
import matplotlib.pyplot as plt

### Loading data

In [2]:
IMG_SIZE = 300
BATCH_SIZE = 32

train_dataset = keras.utils.image_dataset_from_directory(
            '/content/drive/MyDrive/ZINDI-image/train_TF',
            validation_split=0.2,
            subset="training",
            label_mode = 'int',
            seed = 1000,
            image_size=(IMG_SIZE, IMG_SIZE),
            batch_size=BATCH_SIZE)
validation_dataset = keras.utils.image_dataset_from_directory(
            '/content/drive/MyDrive/ZINDI-image/train_TF',
            validation_split=0.2,
            subset="validation",
            label_mode = 'int',
            seed = 1000,
            image_size=(IMG_SIZE, IMG_SIZE),
            batch_size=BATCH_SIZE)

# For performance improvement according to TF
AUTOTUNE = tf.data.AUTOTUNE

train_dataset = train_dataset.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
validation_dataset = validation_dataset.cache().prefetch(buffer_size=AUTOTUNE)

Found 1619 files belonging to 2 classes.
Using 1296 files for training.
Found 1619 files belonging to 2 classes.
Using 323 files for validation.


### Transfer learning

In [3]:
model_folder = 'effV2B0'

base_model = keras.applications.EfficientNetV2B0(
    include_top=False,
    weights='imagenet',
    input_shape=(IMG_SIZE, IMG_SIZE, 3))
base_model.trainable = False
# base_model.summary()

24289280/24274472 [==============================] - 0s 0us/step


In [4]:
data_augmentation = keras.Sequential(
  [
    keras.layers.RandomFlip("horizontal", input_shape=(IMG_SIZE, IMG_SIZE, 3)),
    keras.layers.RandomRotation(0.3),
    keras.layers.RandomZoom(0.3),
  ]
)

In [5]:
inputs = keras.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
# x = data_augmentation(inputs)
x = base_model(inputs, training=False)
x = keras.layers.GlobalAveragePooling2D()(x)
x = keras.layers.Dropout(0.2)(x)
outputs = keras.layers.Dense(1, activation = 'sigmoid')(x)
model = keras.Model(inputs, outputs)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 300, 300, 3)]     0         
                                                                 
 efficientnetv2-b0 (Function  (None, 10, 10, 1280)     5919312   
 al)                                                             
                                                                 
 global_average_pooling2d (G  (None, 1280)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dropout (Dropout)           (None, 1280)              0         
                                                                 
 dense (Dense)               (None, 1)                 1281      
                                                                 
Total params: 5,920,593
Trainable params: 1,281
Non-trainable

In [6]:
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-3),
    loss='BinaryCrossentropy',
    metrics=['accuracy']
)

In [7]:
callbacks_transfer = [
                  keras.callbacks.EarlyStopping(
                      monitor="val_loss",  # Interrupts training when val_loss has stopped improving for 15 epochs
                      mode='min',  # lower the better
                      patience=5
                  )
]

#### Fitting

In [8]:
initial_epochs = 50

history = model.fit(
    train_dataset,
    epochs=initial_epochs,
    validation_data=validation_dataset,
    callbacks=callbacks_transfer
)

Epoch 1/50
41/41 [==============================] - 120s 563ms/step - loss: 0.4762 - accuracy: 0.8171 - val_loss: 0.3039 - val_accuracy: 0.9628
Epoch 2/50
41/41 [==============================] - 3s 64ms/step - loss: 0.2521 - accuracy: 0.9522 - val_loss: 0.2068 - val_accuracy: 0.9567
Epoch 3/50
41/41 [==============================] - 3s 64ms/step - loss: 0.1839 - accuracy: 0.9630 - val_loss: 0.1555 - val_accuracy: 0.9721
Epoch 4/50
41/41 [==============================] - 3s 64ms/step - loss: 0.1483 - accuracy: 0.9761 - val_loss: 0.1242 - val_accuracy: 0.9845
Epoch 5/50
41/41 [==============================] - 3s 64ms/step - loss: 0.1264 - accuracy: 0.9784 - val_loss: 0.1076 - val_accuracy: 0.9845
Epoch 6/50
41/41 [==============================] - 3s 64ms/step - loss: 0.1102 - accuracy: 0.9792 - val_loss: 0.0965 - val_accuracy: 0.9876
Epoch 7/50
41/41 [==============================] - 3s 64ms/step - loss: 0.0971 - accuracy: 0.9838 - val_loss: 0.0859 - val_accuracy: 0.9845
Epoch 8/50

### Fine tuning

In [9]:
base_model.trainable = True

# Freeze all the layers except the last N layers
for layer in base_model.layers[:-3]:
  layer.trainable =  False

# base_model.summary()

In [10]:
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-4),
    loss='BinaryCrossentropy',
    metrics=['accuracy']
)

In [11]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 300, 300, 3)]     0         
                                                                 
 efficientnetv2-b0 (Function  (None, 10, 10, 1280)     5919312   
 al)                                                             
                                                                 
 global_average_pooling2d (G  (None, 1280)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dropout (Dropout)           (None, 1280)              0         
                                                                 
 dense (Dense)               (None, 1)                 1281      
                                                                 
Total params: 5,920,593
Trainable params: 249,601
Non-trainab

#### Fitting

In [12]:
callbacks_fine = [
                  keras.callbacks.EarlyStopping(
                      monitor="val_loss",  # Interrupts training when val_loss has stopped improving for 15 epochs
                      mode='min',  # lower the better
                      patience=5
                  ),
                  keras.callbacks.ModelCheckpoint(
                      filepath='/content/drive/MyDrive/ZINDI-image/checkpoint/'+model_folder,
                      save_best_only=True, # Only saving when the val_loss has improved
                      monitor="val_loss",  
                      mode='min')  # lower the better
]

In [13]:
fine_tune_epochs = 50
total_epochs =  initial_epochs + fine_tune_epochs

history_fine = model.fit(train_dataset,
                         epochs=total_epochs,
                         initial_epoch=history.epoch[-1],
                         validation_data=validation_dataset,
                         callbacks=callbacks_fine)

Epoch 50/100
41/41 [==============================] - 64s 1s/step - loss: 0.0216 - accuracy: 0.9946 - val_loss: 0.0322 - val_accuracy: 0.9876
Epoch 51/100
41/41 [==============================] - 55s 1s/step - loss: 0.0148 - accuracy: 0.9969 - val_loss: 0.0275 - val_accuracy: 0.9907
Epoch 52/100
41/41 [==============================] - 3s 65ms/step - loss: 0.0114 - accuracy: 0.9992 - val_loss: 0.0314 - val_accuracy: 0.9876
Epoch 53/100
41/41 [==============================] - 54s 1s/step - loss: 0.0111 - accuracy: 0.9992 - val_loss: 0.0252 - val_accuracy: 0.9876
Epoch 54/100
41/41 [==============================] - 3s 65ms/step - loss: 0.0081 - accuracy: 1.0000 - val_loss: 0.0263 - val_accuracy: 0.9876
Epoch 55/100
41/41 [==============================] - 3s 65ms/step - loss: 0.0070 - accuracy: 1.0000 - val_loss: 0.0278 - val_accuracy: 0.9876
Epoch 56/100
41/41 [==============================] - 3s 65ms/step - loss: 0.0075 - accuracy: 0.9992 - val_loss: 0.0262 - val_accuracy: 0.9876
Ep